In [47]:
from case_searcher.parse.civil.judgment import JudgmentParser
parser = JudgmentParser("data/second-parse.sqlite")
#parser.sql.drop_tables()
#parser.sql.create_tables()

100%|██████████| 5/5 [00:00<00:00, 2265.48it/s]


In [19]:
# get list of files in s3 bucket
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('court-scraper')
#get list of files in court-scraper bucket
files = []
for obj in bucket.objects.all():
    files.append(obj.key)
len(files)

647

In [20]:
#download file from s3 bucket into memory and parse each html file in the zip
import io
import zipfile
from tqdm import tqdm
for file in tqdm(files):
    if "FTPR" not in file:
        if file.endswith('.zip'):
            obj = s3.Object('court-scraper', files[1])
            body = obj.get()['Body'].read()
            z = zipfile.ZipFile(io.BytesIO(body))
            for filename in z.namelist():
                if filename.endswith('.html'):
                    parser = JudgmentParser("data/second-parse.sqlite")
                    parser.parse(z.open(filename).read())

100%|██████████| 647/647 [06:51<00:00,  1.57it/s]


In [45]:
filename

'D121CV22008073.html'

In [5]:
parser.sql.write("""CREATE TABLE IF NOT EXISTS judgments (
        judgment_id INTEGER PRIMARY KEY AUTOINCREMENT,
        judgment_date STRING,
        judgment_amount STRING,
        judgment_interest STRING,
        judgment_costs STRING,
        judgment_other_amounts STRING,
        judgment_attorney_fees STRING,
        judgment_post_interest_legal_rate STRING,
        judgment_jointly_or_severally STRING,
        judgment_in_favor_of_defendant STRING,
        judgment_possession_of_property_claimed_value_at STRING,
        judgment_is_awarded_to_the STRING,
        judgment_together_with_damages_of STRING,
        judgment_property_value_of_property_sued_for STRING,
        judgment_property_plus_damages_of STRING,
        judgment_property_is_awarded_to_the STRING,
        judgment_property_dismissed_with_prejudice STRING,
        judgment_replevin_detinue_amount STRING,
        judgment_recorded_lien_date STRING,
        judgment_renewed_date STRING,
        judgment_reniewed_lien_date STRING,
        judgment_statisfaction_date STRING,
        case_number STRING,
        FOREIGN KEY (case_number) REFERENCES cases (case_number)
    )""")

In [22]:
import pandas as pd
df = pd.DataFrame(parser.sql.read("""SELECT * FROM judgments"""))

In [29]:
#filter 3 does not eqal None
df = df[df[3] != "None"]
df = df[df[3] != "Legal Rate"]
df = df[df[3] != "$784.73"]
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22


In [27]:
pd.DataFrame(parser.sql.read("""SELECT * FROM attorneys"""))

,0,1,2,3,4,5,6,7,8
0,1,"Friedman, Jeffrey L",100 Owings Court,Suite 4,REISTERSTOWN,MD,21136,plaintiff,D-121-CV-22-007064
1,2,Dollar General Corp. Attn: Payroll,None,None,None,None,None,plaintiff,D-121-CV-22-007064
2,3,"Schlee, David R Jr.",21 Chapel Manor Court,None,PERRY HALL,MD,21128,plaintiff,D-121-CV-22-007118
3,4,"BURKS, JESSICA G",None,None,None,None,None,plaintiff,D-121-CV-22-007118
4,5,"SCHRIER, STEVEN M","Schrier Tolin & Wagman, LLC",700 King Farm Blvd.,ROCKVILLE,MD,20850,plaintiff,D-121-CV-22-007135
...,...,...,...,...,...,...,...,...,...
24670,24671,"Booth, James",None,None,None,None,None,plaintiff,D-121-CV-22-008052
24671,24672,"KLIMA, MICHAEL J","Klima, Peters & Daly, P.A.",8028 Ritchie Hwy,PASADENA,MD,21122,plaintiff,D-121-CV-22-008068
24672,24673,"LEWIS, JOSEPH M",None,None,None,None,None,plaintiff,D-121-CV-22-008068
24673,24674,"GLICK, JAMIE BRETT","Weinstein, Friedman and Friedman",Beco Towers II,OWINGS MILLS,MD,21117,plaintiff,D-121-CV-22-008073


In [28]:
pd.DataFrame(parser.sql.read("""SELECT * FROM cases"""))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,D-121-CV-22-007064,District Court For Allegany County - Civil,Allegany,"DNF Associates, LLC. vs. Ronald Richards",Contract - Large Claims Consumer Debt - Assigned,02/02/2022,Open / Judgment,Affidavit Judgment,None,None,None,None,None,None
1,2,D-121-CV-22-007118,District Court For Allegany County - Civil,Allegany,Gateway Financial Solutions vs. JESSICA BURKS,Contract - Large Claims Consumer Debt - Assigned,02/22/2022,Open,None,None,None,None,None,None,None
2,3,D-121-CV-22-007135,District Court For Allegany County - Civil,Allegany,"Cavalry SPV I, LLC vs. Leonard Kaschenbach",Contract - Large Claims Consumer Debt - Assigned,03/02/2022,Open,None,None,None,None,None,None,None
3,4,D-121-CV-22-007166,District Court For Allegany County - Civil,Allegany,"Portfolio Recovery Associates, LLC vs. Karen D...",Contract - Large Claims Consumer Debt - Assigned,03/16/2022,Open / Judgment,Affidavit Judgment,None,None,None,None,None,None
4,5,D-121-CV-22-007173,District Court For Allegany County - Civil,Allegany,MIDLAND CREDIT MANAGEMENT INC vs. ELIZABETH GU...,Contract - Large Claims Consumer Debt - Assigned,03/17/2022,Closed,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,14171,D-121-CV-22-007985,District Court For Allegany County - Civil,Allegany,"Oliphant Financial, LLC vs. MICHAEL JOHNSON",Contract - Large Claims Consumer Debt - Assigned,11/23/2022,Open / Judgment,Affidavit Judgment,None,None,None,None,None,None
14171,14172,D-121-CV-22-008044,District Court For Allegany County - Civil,Allegany,"Cavalry SPV I, LLC assignee of Citibank, N.A. ...",Contract - Large Claims Consumer Debt - Assigned,12/13/2022,Open / Judgment,Affidavit Judgment,None,None,None,None,None,None
14172,14173,D-121-CV-22-008052,District Court For Allegany County - Civil,Allegany,"Cavalry SPV I, LLC vs. James Booth",Contract - Large Claims Consumer Debt - Assigned,12/14/2022,Open,None,None,None,None,None,None,None
14173,14174,D-121-CV-22-008068,District Court For Allegany County - Civil,Allegany,LVNV FUNDING LLC vs. JOSEPH LEWIS,Contract - Large Claims Consumer Debt - Assigned,12/20/2022,Open,None,None,None,None,None,None,None


In [33]:
file = "/Users/ryanlittle/Downloads/010100000142022-010100005692022--95-files/010100005592022.html"
parser.parse(file)

/Users/ryanlittle/code/packages/case_searcher/case_searcher/parse/civil/judgment.py:370: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")


In [36]:
html = open(file).read()

In [37]:
parser.parse_judgment(html)

TypeError: str.find() takes no keyword arguments

In [44]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
parser.get_value(soup, "Other Amounts")

'$0.00'